In [3]:
from model import *
from data import *
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.3
set_session(tf.Session(config=config))

DATA_DIR = "/data/krf/dataset/ChinaSet_AllFiles"
ROOT_DIR = "/data/krf/model/rsna"

data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')
myGene = trainGenerator(2,DATA_DIR,'CXR_png','mask',data_gen_args,save_to_dir = None)
valGene = trainGenerator(2,DATA_DIR,'val','val_mask',data_gen_args,save_to_dir = None)

model = unet()
model_checkpoint = ModelCheckpoint('unet_lung.hdf5',monitor='val_loss',verbose=1, save_best_only=True)

history = model.fit_generator(myGene,steps_per_epoch=400,epochs=20,validation_data = valGene,validation_steps = 400,callbacks=[model_checkpoint])

Epoch 1/20
Found 66 images belonging to 1 classes.
Found 638 images belonging to 1 classes.
Found 66 images belonging to 1 classes.
Found 638 images belonging to 1 classes.
400/400 [==============================] - 435s 1s/step - loss: 0.6882 - acc: 0.7491 - val_loss: 0.6836 - val_acc: 0.7457

Epoch 00001: val_loss improved from inf to 0.68357, saving model to unet_lung.hdf5
Epoch 2/20
400/400 [==============================] - 432s 1s/step - loss: 0.6793 - acc: 0.7445 - val_loss: 0.6749 - val_acc: 0.7451

Epoch 00002: val_loss improved from 0.68357 to 0.67487, saving model to unet_lung.hdf5
Epoch 3/20
400/400 [==============================] - 428s 1s/step - loss: 0.6706 - acc: 0.7465 - val_loss: 0.6665 - val_acc: 0.7455

Epoch 00003: val_loss improved from 0.67487 to 0.66651, saving model to unet_lung.hdf5
Epoch 4/20
400/400 [==============================] - 428s 1s/step - loss: 0.6621 - acc: 0.7486 - val_loss: 0.6587 - val_acc: 0.7447

Epoch 00004: val_loss improved from 0.66651 t

In [ ]:
CASE_DIR = "/data/krf/dataset/stage_1_train_images/"
def testGenerator2(test_path,num_image,target_size = (256,256),flag_multi_class = False,as_gray = True):
    f_list = os.listdir(test_path)
    f_list.sort()
    for f in f_list[:num_image]:
        #img = io.imread(os.path.join(test_path,f),as_gray = as_gray)
        img = pydicom.read_file(os.path.join(test_path,f)).pixel_array
        img = img / 255
        img = trans.resize(img,target_size)
        img = np.reshape(img,img.shape+(1,)) if (not flag_multi_class) else img
        img = np.reshape(img,(1,)+img.shape)
        yield img
        
testGene2 = testGenerator2(CASE_DIR,len(os.listdir(CASE_DIR)))
model2 = unet()
model2.load_weights("unet_lung.hdf5")
results2 = model2.predict_generator(testGene2,len(os.listdir(CASE_DIR)),verbose=1)

In [ ]:
saveResult(CASE_DIR,"/data/krf/dataset/stage_1_train_masks/",results2)

In [ ]:
TEST_DIR = "/data/krf/dataset/stage_1_test_images/"
testGene3 = testGenerator2(TEST_DIR,50)
model3 = unet()
model3.load_weights("unet_lung.hdf5")
results3 = model3.predict_generator(testGene3,50,verbose=1)

In [ ]:
saveResult(TEST_DIR,"/data/krf/dataset/stage_1_test_masks/",results3)